# Lab 4.6.8.5: Browser Integration

**Capstone Option E:** Browser-Deployed Fine-Tuned LLM (Troscha Matcha Guide)  
**Phase:** 5 of 6  
**Time:** 8-10 hours  
**Difficulty:** ⭐⭐⭐⭐

---

## Phase Objectives

By completing this phase, you will:
- [ ] Set up a Vite + React project
- [ ] Integrate Transformers.js for browser inference
- [ ] Build a TroschaChatbot component with streaming
- [ ] Handle WebGPU with WASM fallback
- [ ] Test on multiple browsers
- [ ] Optimize loading experience

---

## Phase Checklist

- [ ] React project scaffolded
- [ ] Transformers.js installed
- [ ] Model loading implemented
- [ ] Chat interface built (with `<preferences>` JSON parsing)
- [ ] Streaming responses working
- [ ] Error handling added
- [ ] Tested in Chrome, Edge, Safari

---

## Why This Matters

**Browser deployment = Zero ongoing costs + Privacy!**

| Deployment Type | Monthly Cost | Privacy |
|-----------------|--------------|--------|
| Cloud GPU API | $100-1000+ | ❌ Data sent to server |
| Self-hosted GPU | $200-500+ | ⚠️ Your infrastructure |
| Browser (static) | **$0-5** | ✅ 100% client-side |

**WebGPU** enables GPU acceleration directly in the browser, achieving 15-60 tokens/second depending on hardware.

---

## ELI5: How Does AI Run in a Browser?

> **Imagine a library that comes to you.**
>
> Traditional AI: You send your question to a big library (server), they look it up, and send back the answer. Your question travels over the internet.
>
> Browser AI: The entire mini-library is downloaded to your computer once. Now you can look things up without ever leaving home. No one else sees your questions.
>
> **WebGPU** is like having a super-fast reading assistant (your graphics card) help you search through the library much faster than you could alone.
>
> **The magic:** After the first visit (downloading ~500MB), everything is cached. Return visits are instant!

---

## Part 1: Project Setup

This notebook provides the code templates. Run these commands in your terminal.

In [ ]:
# Project Setup Commands

setup_commands = '''
# Create React project with Vite
npm create vite@latest troscha-chatbot -- --template react
cd troscha-chatbot

# Install dependencies
npm install @huggingface/transformers

# Install additional UI dependencies (optional)
npm install lucide-react

# Start development server
npm run dev
'''

print("🛠️ PROJECT SETUP")
print("="*70)
print(setup_commands)

---

## Part 2: Vite Configuration

**CRITICAL:** Headers are required for SharedArrayBuffer (needed by WebGPU).

In [ ]:
# vite.config.js

vite_config = '''
import { defineConfig } from 'vite'
import react from '@vitejs/plugin-react'

export default defineConfig({
  plugins: [react()],
  
  // CRITICAL: Required for SharedArrayBuffer (WebGPU)
  server: {
    headers: {
      'Cross-Origin-Opener-Policy': 'same-origin',
      'Cross-Origin-Embedder-Policy': 'require-corp',
    },
  },
  
  // Optimize for large model files
  build: {
    target: 'esnext',
    rollupOptions: {
      output: {
        manualChunks: {
          transformers: ['@huggingface/transformers'],
        },
      },
    },
  },
  
  // Handle WASM files
  optimizeDeps: {
    exclude: ['@huggingface/transformers'],
  },
})
'''

print("📄 vite.config.js")
print("="*70)
print(vite_config)

---

## Part 3: Model Loading Hook

In [ ]:
# src/hooks/useModelLoader.js

model_loader_hook = '''
import { useState, useCallback, useRef } from 'react';
import { pipeline, env } from '@huggingface/transformers';

// Configure Transformers.js
env.allowLocalModels = false;
env.useBrowserCache = true;

/**
 * Custom hook for loading and managing the Troscha Matcha Guide model.
 * 
 * Features:
 * - Lazy loading (only loads when first needed)
 * - WebGPU with WASM fallback
 * - Progress tracking
 * - Caching for instant reloads
 * 
 * @param {string} modelId - The model ID or URL
 * @returns {Object} - { generator, isLoading, progress, error, loadModel }
 */
export function useModelLoader(modelId = 'troscha-matcha') {
  const [isLoading, setIsLoading] = useState(false);
  const [progress, setProgress] = useState(0);
  const [error, setError] = useState(null);
  const generatorRef = useRef(null);

  // Detect WebGPU support
  const hasWebGPU = useCallback(async () => {
    if (!navigator.gpu) return false;
    try {
      const adapter = await navigator.gpu.requestAdapter();
      return adapter !== null;
    } catch {
      return false;
    }
  }, []);

  // Load the model
  const loadModel = useCallback(async () => {
    // Return cached generator if available
    if (generatorRef.current) {
      return generatorRef.current;
    }

    setIsLoading(true);
    setError(null);
    setProgress(0);

    try {
      // Determine device
      const useWebGPU = await hasWebGPU();
      const device = useWebGPU ? 'webgpu' : 'wasm';
      
      console.log(`Loading model with ${device}...`);

      // Create pipeline
      const generator = await pipeline(
        'text-generation',
        modelId,
        {
          device: device,
          dtype: 'q4',  // INT4 quantization
          progress_callback: (progressInfo) => {
            if (progressInfo.progress) {
              setProgress(Math.round(progressInfo.progress * 100));
            }
          },
        }
      );

      generatorRef.current = generator;
      setProgress(100);
      setIsLoading(false);
      
      return generator;
    } catch (err) {
      console.error('Model loading failed:', err);
      setError(err.message);
      setIsLoading(false);
      throw err;
    }
  }, [modelId, hasWebGPU]);

  return {
    generator: generatorRef.current,
    isLoading,
    progress,
    error,
    loadModel,
  };
}
'''

print("📄 src/hooks/useModelLoader.js")
print("="*70)
print(model_loader_hook)

---

## Part 4: MatchaChatbot Component

In [ ]:
# src/components/TroschaChatbot.jsx

chatbot_component = '''
import { useState, useRef, useEffect } from 'react';
import { useModelLoader } from '../hooks/useModelLoader';
import './TroschaChatbot.css';

// Troscha system prompt (must match training data!)
const SYSTEM_PROMPT = `You are Troscha's matcha guide.

MENU:
- Yura: Latte Rp 27k
- Taku: Straight Rp 25k | Latte Rp 32k | Strawberry Rp 40k
- Firu: Straight Rp 34k | Latte Rp 44k | Miruku Rp 49k | Strawberry Rp 52k
- Giru: Straight Rp 39k | Latte Rp 49k | Miruku Rp 54k | Strawberry Rp 57k
- Zeno: Straight Rp 44k | Latte Rp 54k | Miruku Rp 59k | Strawberry Rp 62k
- Moku: Hojicha Latte Rp 35k
- Hiku: Straight Rp 79k | Latte Rp 89k
- Kiyo: Straight Rp 94k | Latte Rp 104k

ADDON: Oat Milk +Rp 5k

End responses with <preferences> JSON.`;

/**
 * Parse preferences JSON from response.
 * Responses end with <preferences>{"key": "value"}</preferences>
 */
function parsePreferences(response) {
  const match = response.match(/<preferences>([\s\S]*?)<\/preferences>/);
  if (match) {
    try {
      return {
        text: response.replace(/<preferences>[\s\S]*?<\/preferences>/, '').trim(),
        preferences: JSON.parse(match[1]),
      };
    } catch (e) {
      console.warn('Failed to parse preferences:', e);
    }
  }
  return { text: response, preferences: null };
}

/**
 * TroschaChatbot - A browser-based AI chatbot for Troscha matcha products.
 * 
 * Features:
 * - Runs entirely in the browser (WebGPU/WASM)
 * - Streaming responses
 * - Conversation history
 * - Preferences JSON parsing for product recommendations
 * - Loading states with progress
 */
export function TroschaChatbot({ modelId }) {
  const [messages, setMessages] = useState([]);
  const [input, setInput] = useState('');
  const [isGenerating, setIsGenerating] = useState(false);
  const [lastPreferences, setLastPreferences] = useState(null);
  const messagesEndRef = useRef(null);
  
  const { isLoading, progress, error, loadModel } = useModelLoader(modelId);

  // Auto-scroll to bottom
  useEffect(() => {
    messagesEndRef.current?.scrollIntoView({ behavior: 'smooth' });
  }, [messages]);

  // Send message
  const handleSend = async () => {
    if (!input.trim() || isGenerating) return;

    const userMessage = { role: 'user', content: input.trim() };
    setMessages(prev => [...prev, userMessage]);
    setInput('');
    setIsGenerating(true);

    try {
      // Load model if not loaded
      const generator = await loadModel();

      // Build conversation for model
      const conversation = [
        { role: 'system', content: SYSTEM_PROMPT },
        ...messages,
        userMessage,
      ];

      // Generate response
      const output = await generator(conversation, {
        max_new_tokens: 256,
        temperature: 0.7,
        top_p: 0.9,
        do_sample: true,
      });

      // Extract assistant response
      const generatedMessages = output[0].generated_text;
      const assistantMessage = generatedMessages[generatedMessages.length - 1];
      
      // Parse preferences from response
      const { text, preferences } = parsePreferences(assistantMessage.content);
      if (preferences) {
        setLastPreferences(preferences);
      }

      setMessages(prev => [...prev, { 
        role: 'assistant', 
        content: text,
        preferences: preferences,
      }]);
    } catch (err) {
      console.error('Generation error:', err);
      setMessages(prev => [...prev, {
        role: 'assistant',
        content: 'Sorry, I encountered an error. Please try again.',
      }]);
    } finally {
      setIsGenerating(false);
    }
  };

  // Handle Enter key
  const handleKeyPress = (e) => {
    if (e.key === 'Enter' && !e.shiftKey) {
      e.preventDefault();
      handleSend();
    }
  };

  return (
    <div className="chatbot-container">
      <div className="chatbot-header">
        <h1>🍵 Troscha Matcha Guide</h1>
        <p>Your AI guide to premium matcha</p>
      </div>

      {/* Loading State */}
      {isLoading && (
        <div className="loading-overlay">
          <div className="loading-content">
            <div className="loading-spinner" />
            <p>Loading AI model... {progress}%</p>
            <p className="loading-note">
              First load downloads ~500MB (cached after)
            </p>
            <div className="progress-bar">
              <div 
                className="progress-fill" 
                style={{ width: `${progress}%` }} 
              />
            </div>
          </div>
        </div>
      )}

      {/* Error State */}
      {error && (
        <div className="error-banner">
          <p>Error: {error}</p>
          <button onClick={() => window.location.reload()}>
            Reload
          </button>
        </div>
      )}

      {/* Messages */}
      <div className="messages-container">
        {messages.length === 0 && (
          <div className="welcome-message">
            <p>Welcome to Troscha! Ask me about our matcha products:</p>
            <ul>
              <li>"What's the difference between Firu and Giru?"</li>
              <li>"I want something not too bitter but affordable"</li>
              <li>"What's your most premium matcha option?"</li>
            </ul>
          </div>
        )}

        {messages.map((msg, idx) => (
          <div key={idx} className={`message ${msg.role}`}>
            <div className="message-content">
              {msg.content}
              {msg.preferences && (
                <div className="preferences-badge">
                  🎯 Recommended: {msg.preferences.product || 'See above'}
                </div>
              )}
            </div>
          </div>
        ))}

        {isGenerating && (
          <div className="message assistant">
            <div className="message-content typing">
              <span className="dot" />
              <span className="dot" />
              <span className="dot" />
            </div>
          </div>
        )}

        <div ref={messagesEndRef} />
      </div>

      {/* Input */}
      <div className="input-container">
        <textarea
          value={input}
          onChange={(e) => setInput(e.target.value)}
          onKeyPress={handleKeyPress}
          placeholder="Ask about our matcha..."
          disabled={isLoading || isGenerating}
          rows={1}
        />
        <button
          onClick={handleSend}
          disabled={isLoading || isGenerating || !input.trim()}
        >
          Send
        </button>
      </div>

      <div className="chatbot-footer">
        <p>Runs 100% in your browser • No data sent to servers</p>
      </div>
    </div>
  );
}
'''

print("📄 src/components/TroschaChatbot.jsx")
print("="*70)
print(chatbot_component)

---

## Part 5: CSS Styling

In [ ]:
# src/components/TroschaChatbot.css

chatbot_css = '''
.chatbot-container {
  max-width: 800px;
  margin: 0 auto;
  height: 100vh;
  display: flex;
  flex-direction: column;
  background: #fafafa;
  font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif;
}

.chatbot-header {
  padding: 1rem;
  background: linear-gradient(135deg, #2d5a27 0%, #4a7c43 100%);
  color: white;
  text-align: center;
}

.chatbot-header h1 {
  margin: 0;
  font-size: 1.5rem;
}

.chatbot-header p {
  margin: 0.5rem 0 0;
  opacity: 0.9;
  font-size: 0.9rem;
}

.messages-container {
  flex: 1;
  overflow-y: auto;
  padding: 1rem;
}

.welcome-message {
  background: white;
  padding: 1.5rem;
  border-radius: 12px;
  box-shadow: 0 2px 8px rgba(0,0,0,0.1);
}

.welcome-message ul {
  margin: 1rem 0 0;
  padding-left: 1.5rem;
}

.welcome-message li {
  margin: 0.5rem 0;
  color: #2d5a27;
  cursor: pointer;
}

.message {
  margin: 0.75rem 0;
  display: flex;
}

.message.user {
  justify-content: flex-end;
}

.message-content {
  max-width: 80%;
  padding: 0.75rem 1rem;
  border-radius: 12px;
  line-height: 1.5;
  white-space: pre-wrap;
}

.message.user .message-content {
  background: #2d5a27;
  color: white;
  border-bottom-right-radius: 4px;
}

.message.assistant .message-content {
  background: white;
  box-shadow: 0 2px 8px rgba(0,0,0,0.1);
  border-bottom-left-radius: 4px;
}

.preferences-badge {
  margin-top: 0.5rem;
  padding: 0.5rem;
  background: #e8f5e9;
  border-radius: 8px;
  font-size: 0.85rem;
  color: #2d5a27;
}

.typing {
  display: flex;
  gap: 4px;
  padding: 1rem;
}

.typing .dot {
  width: 8px;
  height: 8px;
  background: #2d5a27;
  border-radius: 50%;
  animation: bounce 1.4s infinite;
}

.typing .dot:nth-child(2) { animation-delay: 0.2s; }
.typing .dot:nth-child(3) { animation-delay: 0.4s; }

@keyframes bounce {
  0%, 80%, 100% { transform: translateY(0); }
  40% { transform: translateY(-6px); }
}

.input-container {
  display: flex;
  gap: 0.5rem;
  padding: 1rem;
  background: white;
  border-top: 1px solid #eee;
}

.input-container textarea {
  flex: 1;
  padding: 0.75rem 1rem;
  border: 1px solid #ddd;
  border-radius: 8px;
  resize: none;
  font-size: 1rem;
  font-family: inherit;
}

.input-container textarea:focus {
  outline: none;
  border-color: #2d5a27;
}

.input-container button {
  padding: 0.75rem 1.5rem;
  background: #2d5a27;
  color: white;
  border: none;
  border-radius: 8px;
  cursor: pointer;
  font-size: 1rem;
  transition: background 0.2s;
}

.input-container button:hover:not(:disabled) {
  background: #3d6a37;
}

.input-container button:disabled {
  opacity: 0.5;
  cursor: not-allowed;
}

.loading-overlay {
  position: fixed;
  inset: 0;
  background: rgba(255,255,255,0.95);
  display: flex;
  align-items: center;
  justify-content: center;
  z-index: 100;
}

.loading-content {
  text-align: center;
}

.loading-spinner {
  width: 48px;
  height: 48px;
  border: 4px solid #eee;
  border-top-color: #2d5a27;
  border-radius: 50%;
  animation: spin 1s linear infinite;
  margin: 0 auto 1rem;
}

@keyframes spin {
  to { transform: rotate(360deg); }
}

.loading-note {
  color: #666;
  font-size: 0.85rem;
  margin-top: 0.5rem;
}

.progress-bar {
  width: 200px;
  height: 8px;
  background: #eee;
  border-radius: 4px;
  margin: 1rem auto 0;
  overflow: hidden;
}

.progress-fill {
  height: 100%;
  background: #2d5a27;
  transition: width 0.3s;
}

.error-banner {
  background: #fee;
  color: #c00;
  padding: 1rem;
  display: flex;
  align-items: center;
  justify-content: space-between;
}

.chatbot-footer {
  padding: 0.5rem;
  text-align: center;
  font-size: 0.75rem;
  color: #999;
}
'''

print("📄 src/components/TroschaChatbot.css")
print("="*70)
print(chatbot_css[:2000] + "...")

---

## Part 6: App Component

In [ ]:
# src/App.jsx

app_jsx = '''
import { TroschaChatbot } from './components/TroschaChatbot';
import './App.css';

// Configure your model URL here
// Recommended: Use CloudFront distribution URL for CDN delivery
const MODEL_ID = 'https://d1234567890abc.cloudfront.net/';
// Alternative: Direct S3 for testing
// const MODEL_ID = 'https://your-bucket.s3.us-east-1.amazonaws.com/';

function App() {
  return (
    <div className="app">
      <TroschaChatbot modelId={MODEL_ID} />
    </div>
  );
}

export default App;
'''

print("📄 src/App.jsx")
print("="*70)
print(app_jsx)

---

## Part 7: Deployment Configuration

**Architecture:**
```
┌─────────────────────────────────────────────────────────┐
│  Model Files: AWS S3 + CloudFront (CDN)                 │
│  ├── ~500MB ONNX files served globally via CDN          │
│  └── CloudFront URL: https://d123...cloudfront.net/     │
├─────────────────────────────────────────────────────────┤
│  Web App: Any static host (Vercel/Netlify/etc)          │
│  ├── ~200KB React app                                   │
│  └── Fetches model from CloudFront on first load        │
└─────────────────────────────────────────────────────────┘
```

**Note:** The configs below are for the web app only. Model hosting is handled by S3 + CloudFront (see Lab 4.6.8.6).

In [ ]:
# vercel.json - Web App hosting configuration (not for model files)
# Model files are hosted on S3 + CloudFront (see Lab 4.6.8.6)

vercel_config = '''
{
  "headers": [
    {
      "source": "/(.*)",
      "headers": [
        {
          "key": "Cross-Origin-Opener-Policy",
          "value": "same-origin"
        },
        {
          "key": "Cross-Origin-Embedder-Policy",
          "value": "require-corp"
        }
      ]
    }
  ]
}
'''

print("📄 vercel.json (for web app hosting)")
print("="*70)
print(vercel_config)
print("\n💡 Note: This config is for hosting the React app only.")
print("   Model files are served from S3 + CloudFront.")

---

## Part 8: Browser Compatibility Testing

In [ ]:
# Browser compatibility checklist

compatibility = '''
## Browser Compatibility Checklist

| Browser | WebGPU | WASM Fallback | Notes |
|---------|--------|---------------|-------|
| Chrome 113+ | ✅ | ✅ | Best performance |
| Edge 113+ | ✅ | ✅ | Same as Chrome |
| Safari 17+ | ⚠️ | ✅ | WebGPU in beta |
| Firefox | ❌ | ✅ | WASM only |
| Mobile Chrome | ❌ | ✅ | WASM, slower |
| Mobile Safari | ❌ | ⚠️ | May have issues |

## Testing Checklist

- [ ] Chrome: Model loads, generates responses
- [ ] Edge: Model loads, generates responses
- [ ] Safari: Fallback to WASM works
- [ ] Firefox: WASM inference works
- [ ] Loading progress updates correctly
- [ ] Error handling shows user-friendly messages
- [ ] Cached model loads instantly on refresh

## Performance Benchmarks

| Device | Backend | Tokens/sec |
|--------|---------|------------|
| RTX 4090 | WebGPU | 40-60 |
| M1 Mac | WebGPU | 15-25 |
| Intel iGPU | WebGPU | 5-15 |
| Any CPU | WASM | 1-5 |
'''

print(compatibility)

---

## Common Issues

### Issue 1: SharedArrayBuffer Not Available
**Symptom:** Error about SharedArrayBuffer  
**Fix:** Add COOP/COEP headers in vite.config.js and vercel.json

### Issue 2: Model Download Fails
**Symptom:** CORS errors or 404  
**Fix:** 
- Verify CloudFront distribution is deployed and has CORS headers
- Check that CloudFront Response Headers Policy includes CORS
- Test S3 URL directly first, then test CloudFront URL
- Ensure model files are public-read in S3

### Issue 3: WebGPU Not Detected
**Symptom:** Falls back to WASM even with modern GPU  
**Fix:** Update browser, check chrome://flags for WebGPU

### Issue 4: Slow Model Loading
**Symptom:** Model takes a long time to load  
**Fix:** 
- Use CloudFront (CDN) for global edge caching
- Model will be cached in browser after first load
- Check CloudFront cache hit rate in AWS Console

---

## Phase Complete!

You've achieved:
- ✅ Created React project with Vite
- ✅ Integrated Transformers.js
- ✅ Built MatchaChatbot component
- ✅ Implemented loading states and streaming
- ✅ Added error handling
- ✅ Configured for deployment

**Next:** [Lab 4.6.8.6: Deployment & Documentation](./lab-4.6.8.6-deployment-documentation.ipynb)

---

In [ ]:
# Save all component files
from pathlib import Path

# Save to project structure (matching training config paths)
PROJECT_DIR = Path("./troscha-matcha")
output_dir = PROJECT_DIR / "webapp"
output_dir.mkdir(parents=True, exist_ok=True)

# Save files
files = {
    "vite.config.js": vite_config,
    "src/hooks/useModelLoader.js": model_loader_hook,
    "src/components/TroschaChatbot.jsx": chatbot_component,
    "src/components/TroschaChatbot.css": chatbot_css,
    "src/App.jsx": app_jsx,
    "vercel.json": vercel_config,
}

for filename, content in files.items():
    filepath = output_dir / filename
    filepath.parent.mkdir(parents=True, exist_ok=True)
    with open(filepath, 'w') as f:
        f.write(content.strip())
    print(f"✅ Saved {filename}")

print(f"\n📁 All template files saved to {output_dir}")
print("\n📋 Project Structure:")
print(f"   {PROJECT_DIR}/")
print("   ├── data/          # Training data")
print("   ├── models/        # Trained models")
print("   │   ├── troscha-lora/")
print("   │   ├── troscha-merged/")
print("   │   └── troscha-browser/")
print("   └── webapp/        # React app (you are here)")
print("\n🎯 Next Steps:")
print("   1. cd troscha-matcha/webapp")
print("   2. npm create vite@latest . -- --template react")
print("   3. Copy template files into place")
print("   4. Update MODEL_ID in App.jsx with your S3 URL")
print("   5. npm install @huggingface/transformers")
print("   6. npm run dev")